___________ 

In [ ]:
pip install pymysql

In [1]:
import os
import time
import pandas as pd
import mysql.connector
import gzip
from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

from datetime import datetime, timedelta
datetime.today().strftime('Data: %Y-%m-%d Hora: %H:%m')

'Data: 2025-03-24 Hora: 11:03'

In [2]:
# #CAMINHO DA PASTA COM OS ARQUIVOS
arquivos = os.listdir("//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Estoques")
df_csv_agis = pd.DataFrame()
len(arquivos)

67

#### E se eu fizer os paths de arquivos separados por custodiante? De manhã eu só rodaria o path de Singulare e não precisaria mover os arquivos da pasta na rede

### Excluir esses arquivos, pois já atualizamos no singulare (estoque)

* 33499726000170_Estoque_BRISTOL FIDC MULTISSETORIAL_001
* 51660048000167_Estoque_APG FIDC_001
* 41520452000181_Estoque_AGROFORTE FIDC_001

# CRIA DATAFRAME AGIS

In [3]:
import pandas as pd

# Encontra o arquivo AGIS
csv_agis = list(filter(lambda x: 'ÁGIS FIDC' in x, arquivos))

# Inicializa o DataFrame vazio
df_csv_agis = pd.DataFrame()

try:
    # Lê o arquivo CSV, ignorando linhas problemáticas
    df_csv_agis = pd.read_csv(
        f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Estoques/{csv_agis[0]}',
        delimiter=';',
        encoding='latin-1',
        dtype='str',
        on_bad_lines='skip'  # Ignora linhas com erros
    )

    # Filtra apenas as colunas necessárias
    colunas_desejadas = [
        'NOME_FUNDO', 'DOC_FUNDO', 'NOME_CEDENTE', 'DOC_CEDENTE', 'NOME_SACADO',
        'DOC_SACADO', 'SEU_NUMERO', 'NU_DOCUMENTO', 'TIPO_RECEBIVEL',
        'VALOR_NOMINAL', 'VALOR_PRESENTE', 'VALOR_AQUISICAO', 'VALOR_PDD',
        'FAIXA_PDD', 'DATA_REFERENCIA', 'DATA_VENCIMENTO_ORIGINAL',
        'DATA_VENCIMENTO_AJUSTADA', 'DATA_EMISSAO', 'DATA_AQUISICAO',
        'TAXA_CESSAO', 'TX_RECEBIVEL', 'COOBRIGACAO'
    ]
    df_csv_agis = df_csv_agis[colunas_desejadas]

    # Renomeia as colunas
    df_csv_agis.columns = [
        'NmFundo', 'DocFundo', 'NmCedente', 'DocCedente', 'NmSacado', 'DocSacado',
        'Operacao', 'NuDocumento', 'TpRecebivel', 'VlrNominal', 'VlrPresente',
        'VlrAquisicao', 'VlrPDD', 'FaixaPDD', 'DtReferencia', 'DtVencimento',
        'DtVencimentoAjustada', 'DtEmissao', 'DtAquisicao', 'TaxaCessao',
        'TaxaRecebivel', 'Coobrigacao'
    ]

    # Converte as colunas de data
    date_columns = ['DtReferencia', 'DtVencimento', 'DtVencimentoAjustada', 'DtEmissao', 'DtAquisicao']
    for col in date_columns:
        df_csv_agis[col] = pd.to_datetime(df_csv_agis[col], format="%d/%m/%Y", errors='coerce')

    # Função para tratar valores numéricos
    def transformar_valor(valor):
        if pd.isna(valor):
            return valor
        valor_str = str(valor)
        valor_str = valor_str.replace('.', '').replace(',', '.')
        return float(valor_str)

    # Converte as colunas numéricas
    columns_to_convert = ['VlrNominal', 'VlrPresente', 'VlrAquisicao', 'VlrPDD', 'TaxaCessao', 'TaxaRecebivel']
    for column in columns_to_convert:
        df_csv_agis[column] = df_csv_agis[column].apply(transformar_valor)

except Exception as e:
    print(f"Erro ao processar o arquivo: {e}")

In [4]:
df_csv_agis[['VlrNominal', 'VlrPresente' , 'VlrAquisicao', 'VlrPDD']]

,VlrNominal,VlrPresente,VlrAquisicao,VlrPDD
0,63265.56,63265.56,61106.28,63265.56
1,210748.00,210748.00,204215.48,210748.00
2,24400.00,24400.00,22965.67,24400.00
3,57264.16,57264.16,55306.22,57264.16
4,24400.00,24400.00,22148.50,24400.00
...,...,...,...,...
11955,20615.55,19394.39,19243.16,0.00
11956,2120.20,1994.62,1979.07,0.00
11957,28444.41,28185.28,28086.24,140.93
11958,3451.80,3420.35,3408.33,17.10


## ESTOQUE OUTROS FUNDOS

In [5]:
## CRIA DATAFRAME COM OS DADOS DE ESTOQUE DA PASTA/ OUTROS FUNDOS

string = ",".join(str(element) for element in csv_agis)
arquivos = list(filter(lambda x: x != str(string), arquivos))  # Dispensa arquivo AGIS

stop = 0
df = pd.DataFrame()

while stop < len(arquivos):
    
    # Leitura do arquivo com on_bad_lines='skip' para ignorar linhas problemáticas
    estoque = pd.read_csv(
        f'//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Estoques/{arquivos[stop]}',
        delimiter=';',
        encoding='latin-1',
        dtype='str',
        on_bad_lines='skip'  # Ignora automaticamente as linhas que causariam erro
    )
    
    df = pd.concat([df, estoque], ignore_index=True)
    
    stop += 1

# Seleção das colunas desejadas
df_estoque = df[['NOME_FUNDO', 'DOC_FUNDO', 'NOME_CEDENTE', 'DOC_CEDENTE', 'NOME_SACADO',
                 'DOC_SACADO', 'SEU_NUMERO', 'NU_DOCUMENTO', 'TIPO_RECEBIVEL',
                 'VALOR_NOMINAL', 'VALOR_PRESENTE', 'VALOR_AQUISICAO', 'VALOR_PDD',
                 'FAIXA_PDD', 'DATA_REFERENCIA', 'DATA_VENCIMENTO_ORIGINAL',
                 'DATA_VENCIMENTO_AJUSTADA', 'DATA_EMISSAO', 'DATA_AQUISICAO', 'COOBRIGACAO']]


In [6]:
# Renomeia as colunas
df_estoque.columns = ['NmFundo', 'DocFundo', 'NmCedente', 'DocCedente', 'NmSacado', 'DocSacado', 'Operacao', 
                      'NuDocumento', 'TpRecebivel', 'VlrNominal', 'VlrPresente', 'VlrAquisicao', 'VlrPDD', 
                      'FaixaPDD', 'DtReferencia', 'DtVencimento', 'DtVencimentoAjustada', 'DtEmissao', 
                      'DtAquisicao', 'Coobrigacao']

# Alterando o tipo das colunas de data com tratamento de erros
date_columns = ['DtReferencia', 'DtVencimento', 'DtVencimentoAjustada', 'DtEmissao', 'DtAquisicao']

df_estoque[date_columns] = df_estoque[date_columns].apply(
    pd.to_datetime, format="%d/%m/%Y", errors='coerce'  # Converte valores inválidos para NaT
)

# Tratando dados numéricos
columns_to_convert = ['VlrNominal', 'VlrPresente', 'VlrAquisicao', 'VlrPDD']

def transformar_valor(valor):
    try:
        valor_str = str(valor).replace('.', '').replace(',', '.')
        return float(valor_str)
    except ValueError:  # Captura valores não numéricos e retorna NaN
        return pd.NA

for column in columns_to_convert:
    df_estoque[column] = df_estoque[column].apply(transformar_valor)

# Remove duplicados
df_estoque = df_estoque.drop_duplicates()

# Uni os DataFrames
df_estoque = pd.concat([df_estoque, df_csv_agis])

# Exibe informações
print(f"{max(df_estoque['DtReferencia'])} Data Referência \n{min(df_estoque['DtReferencia'])} \n{len(df_estoque['NmFundo'].unique())} Carregados \n{len(df_estoque['NmFundo'])} Total de linhas")


C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_5808\2635697445.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estoque[date_columns] = df_estoque[date_columns].apply(
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_5808\2635697445.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estoque[column] = df_estoque[column].apply(transformar_valor)


2025-03-20 00:00:00 Data Referência 
2025-03-18 00:00:00 
67 Carregados 
1233424 Total de linhas


In [7]:
last_date = df_estoque['DtReferencia'].max()
print(f"Last date in dataframe df_estoque: {last_date}")

Last date in dataframe df_estoque: 2025-03-20 00:00:00


In [8]:
df_estoque.columns

Index(['NmFundo', 'DocFundo', 'NmCedente', 'DocCedente', 'NmSacado',
       'DocSacado', 'Operacao', 'NuDocumento', 'TpRecebivel', 'VlrNominal',
       'VlrPresente', 'VlrAquisicao', 'VlrPDD', 'FaixaPDD', 'DtReferencia',
       'DtVencimento', 'DtVencimentoAjustada', 'DtEmissao', 'DtAquisicao',
       'Coobrigacao', 'TaxaCessao', 'TaxaRecebivel'],
      dtype='object')

In [9]:
# Remover as colunas 'TaxaCessao' e 'TaxaRecebivel'
df_estoque = df_estoque.drop(columns=['TaxaCessao', 'TaxaRecebivel'])

# Exibir as primeiras linhas para confirmar
print(df_estoque.head())

    NmFundo            DocFundo                                 NmCedente  \
0  FIDC DFC  29.109.316/0001-06      ENERTEX IND. E COM. DE BATERIAS LTDA   
1  FIDC DFC  29.109.316/0001-06  NUTRIWAY FOODS INDUSTRIA ALIMENTICIA LTD   
2  FIDC DFC  29.109.316/0001-06  NUTRIWAY FOODS INDUSTRIA ALIMENTICIA LTD   
3  FIDC DFC  29.109.316/0001-06  NUTRIWAY FOODS INDUSTRIA ALIMENTICIA LTD   
4  FIDC DFC  29.109.316/0001-06  NUTRIWAY FOODS INDUSTRIA ALIMENTICIA LTD   

           DocCedente                                 NmSacado  \
0  14.605.901/0001-11                   BAT FLEX BATERIAS LTDA   
1  14.906.839/0001-06                  ARMAZEM MATEUS S A 4003   
2  14.906.839/0001-06        PACALUZ COMERCIO E LOGISTICA LTDA   
3  14.906.839/0001-06  PERFORME FLAVOR INDUSTRIA E COMERCIO DE   
4  14.906.839/0001-06  PERFORME FLAVOR INDUSTRIA E COMERCIO DE   

            DocSacado                  Operacao NuDocumento TpRecebivel  \
0  11.364.574/0001-65  R00000026545 01 01 00001       12953   Dup

# Tratativa nomes dos Fundos 

In [10]:
df_estoque.loc[df_estoque['NmFundo'] == 'ÁGIS 2 FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_estoque.loc[df_estoque['NmFundo'] == 'ÁGIS FIDC', 'NmFundo'] = 'FIDC AGIS'
df_estoque.loc[df_estoque['NmFundo'] == 'AGIS FIDC NAO PADRONIZADO - TRANSFERIDO', 'NmFundo'] = 'FIDC AGIS'
df_estoque.loc[df_estoque['NmFundo'] == 'AGROFORTE FIDC', 'NmFundo'] = 'FIDC AGROFORTE'
df_estoque.loc[df_estoque['NmFundo'] == 'ALBAREDO FIDC', 'NmFundo'] = 'FIDC ALBAREDO'
df_estoque.loc[df_estoque['NmFundo'] == 'APG FIDC', 'NmFundo'] = 'FIDC APG'
df_estoque.loc[df_estoque['NmFundo'] == 'SC FIDC RESPONSABILIDADE ILIMITADA', 'NmFundo'] = 'FIDC SC'
df_estoque.loc[df_estoque['NmFundo'] == 'Prime Agro FIDC RL', 'NmFundo'] = 'FIDC PRIME AGRO'
df_estoque.loc[df_estoque['NmFundo'] == 'IPE FIDC NP', 'NmFundo'] = 'FIDC IPE'
df_estoque.loc[df_estoque['NmFundo'] == 'SDL - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC SDL'
df_estoque.loc[df_estoque['NmFundo'] == 'BLUE ROCKET FIDC', 'NmFundo'] = 'FIDC B ROCKET'
df_estoque.loc[df_estoque['NmFundo'] == 'K-FINANCE FIDC NP', 'NmFundo'] = 'FIDC K-FINANCE'
df_estoque.loc[df_estoque['NmFundo'] == 'Z INVEST FIDC', 'NmFundo'] = 'FIDC Z INVEST'
df_estoque.loc[df_estoque['NmFundo'] == 'GLOBAL FUTURA FIDC RL FIDC', 'NmFundo'] = 'FIDC GLOBAL FUTURA'
df_estoque.loc[df_estoque['NmFundo'] == 'BASA FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BASA'
df_estoque.loc[df_estoque['NmFundo'] == 'AF6 FIDC NP', 'NmFundo'] = 'FIDC AF6'
df_estoque.loc[df_estoque['NmFundo'] == 'VERGINIA FIDC', 'NmFundo'] = 'FIDC VERGINIA'
df_estoque.loc[df_estoque['NmFundo'] == 'BELL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC BELL'
df_estoque.loc[df_estoque['NmFundo'] == 'CREDIAL BANK PAN FIDC', 'NmFundo'] = 'FIDC CREDIAL'
df_estoque.loc[df_estoque['NmFundo'] == 'VISHNU FIDC', 'NmFundo'] = 'FIDC VISHNU'
df_estoque.loc[df_estoque['NmFundo'] == 'CREDILOG FIDC', 'NmFundo'] = 'FIDC CREDILOG'
df_estoque.loc[df_estoque['NmFundo'] == 'BONTEMPO FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BONTEMPO'
df_estoque.loc[df_estoque['NmFundo'] == 'DFC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NP', 'NmFundo'] = 'FIDC DFC'
df_estoque.loc[df_estoque['NmFundo'] == 'KERDOS FIDC', 'NmFundo'] = 'FIDC KERDOS'
df_estoque.loc[df_estoque['NmFundo'] == 'DBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC DBANK'
df_estoque.loc[df_estoque['NmFundo'] == 'PINPAG FIDC', 'NmFundo'] = 'FIDC PINPAG'
df_estoque.loc[df_estoque['NmFundo'] == 'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC FUTURO CAPITAL'
df_estoque.loc[df_estoque['NmFundo'] == 'AGROCETE FIDC NP', 'NmFundo'] = 'FIDC AGROCETE'
df_estoque.loc[df_estoque['NmFundo'] == 'VELSO - FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO-PADRONIZADOS', 'NmFundo'] = 'FIDC VELSO'

df_estoque.loc[df_estoque['NmFundo'] == 'PAY CARGO FIDC NP', 'NmFundo'] = 'FIDC PAY CARGO'
df_estoque.loc[df_estoque['NmFundo'] == 'GREENWOOD FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS NAO PADRONIZADOS', 'NmFundo'] = 'FIDC GREENWOOD'
df_estoque.loc[df_estoque['NmFundo'] == 'CAPITALIZA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CAPITALIZA'

df_estoque.loc[df_estoque['NmFundo'] == 'NINE CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC NINE CAPITAL'
df_estoque.loc[df_estoque['NmFundo'] == 'BEFIC FIDC DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC BEFIC'
df_estoque.loc[df_estoque['NmFundo'] == 'LIBRA CONSIGNADO FIDC DE RL', 'NmFundo'] = 'FIDC LIBRA CONSIGNADO'
df_estoque.loc[df_estoque['NmFundo'] == 'MASTRENN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MASTRENN'
df_estoque.loc[df_estoque['NmFundo'] == 'CREDILOG II - FIDC', 'NmFundo'] = 'FIDC CREDILOG II'
df_estoque.loc[df_estoque['NmFundo'] == 'F2 BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC F2 BANK'
df_estoque.loc[df_estoque['NmFundo'] == 'UKF FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC UKF'
df_estoque.loc[df_estoque['NmFundo'] == 'MF GROUP FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC MF GROUP'
df_estoque.loc[df_estoque['NmFundo'] == 'NR11 FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC NR11'
df_estoque.loc[df_estoque['NmFundo'] == 'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA  NIX AUDITORES INDEPENDENTES', 'NmFundo'] = 'FIDC VITTRA'
df_estoque.loc[df_estoque['NmFundo'] == 'ANIL FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANIL'
df_estoque.loc[df_estoque['NmFundo'] == 'ANVERES  FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ANVERES'
df_estoque.loc[df_estoque['NmFundo'] == 'CONDOBEM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC CONDOBEM'
df_estoque.loc[df_estoque['NmFundo'] == '3RD - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC 3RD'
df_estoque.loc[df_estoque['NmFundo'] == 'SMT AGRO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC SMT AGRO'
df_estoque.loc[df_estoque['NmFundo'] == 'BR8 FIDC', 'NmFundo'] = 'FIDC BR8'
df_estoque.loc[df_estoque['NmFundo'] == 'RHB CRED FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC RHB'
df_estoque.loc[df_estoque['NmFundo'] == 'MALBEC FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS', 'NmFundo'] = 'FIDC MALBEC'
df_estoque.loc[df_estoque['NmFundo'] == 'USECORP CATÁLISE FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS SEGMENTO COMERCIAL DE RESP LIMITADA', 'NmFundo'] = 'FIDC USECORP CATALISE'
df_estoque.loc[df_estoque['NmFundo'] == 'LEX CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC LEX'
df_estoque.loc[df_estoque['NmFundo'] == 'TERTON FUNDO DE INVESTIMENTO DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TERTON'
df_estoque.loc[df_estoque['NmFundo'] == 'FIDC BOAZ', 'NmFundo'] = 'FIDC BOAZ'
df_estoque.loc[df_estoque['NmFundo'] == 'TORONTO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC TORONTO'
df_estoque.loc[df_estoque['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL', 'NmFundo'] = 'FIDC BRISTOL'
df_estoque.loc[df_estoque['NmFundo'] == 'LOGFIN FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS', 'NmFundo'] = 'FIDC LOGFIN'
df_estoque.loc[df_estoque['NmFundo'] == 'YELLOW CAPITAL FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC YELLOW'
df_estoque.loc[df_estoque['NmFundo'] == 'EJM FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC EJM'
df_estoque.loc[df_estoque['NmFundo'] == 'MENEGOTTI FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO', 'NmFundo'] = 'FIDC MENEGOTTI'
df_estoque.loc[df_estoque['NmFundo'] == 'ZAB LEGACY FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC ZAB LEGACY'
df_estoque.loc[df_estoque['NmFundo'] == 'SAGEL BANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS DE RESPONSABILIDADE LIMITADA', 'NmFundo'] = 'FIDC SAGEL BANK'

print(f"{df_estoque.NmFundo.sort_values(ascending= True).drop_duplicates()} \n {len(df_estoque.NmFundo.drop_duplicates())}")

80427         FIDC 3G BANK
72325             FIDC 3RD
1054415           FIDC AF6
11959            FIDC AGIS
1042642      FIDC AGROCETE
                ...       
1042829        FIDC VISHNU
1169005        FIDC VITTRA
1220731        FIDC YELLOW
1108497      FIDC Z INVEST
1220840    FIDC ZAB LEGACY
Name: NmFundo, Length: 67, dtype: object 
 67


In [11]:
df_estoque.NmFundo[942068]

'FIDC TOPCRED'

In [12]:
df_estoque[['DtReferencia']].drop_duplicates()

,DtReferencia
0,2025-03-20
1220781,2025-03-18


In [13]:
df_estoque[['DtReferencia','NmFundo','DocFundo']].drop_duplicates().sort_values("DtReferencia",ascending=False)

,DtReferencia,NmFundo,DocFundo
0,2025-03-20,FIDC DFC,29.109.316/0001-06
1005,2025-03-20,FIDC IPE,30.142.432/0001-06
1072964,2025-03-20,FIDC USECORP CATALISE,52.997.828/0001-60
1106554,2025-03-20,FIDC ALBAREDO,53.073.677/0001-16
1108394,2025-03-20,FIDC Z INVEST,53.160.371/0001-05
...,...,...,...
1042684,2025-03-20,FIDC CREDIAL,48.928.987/0001-35
1042520,2025-03-20,FIDC AGROCETE,48.348.391/0001-66
0,2025-03-20,FIDC AGIS,35.689.601/0001-38
1220781,2025-03-18,FIDC ZAB LEGACY,58.495.913/0001-06


In [14]:
print(df_estoque[['NmFundo', 'DtReferencia']].drop_duplicates().head(64).to_string())

                       NmFundo DtReferencia
0                     FIDC DFC   2025-03-20
1005                  FIDC IPE   2025-03-20
2568               FIDC FERPAR   2025-03-20
5094            FIDC GREENWOOD   2025-03-20
5230            FIDC PAY CARGO   2025-03-20
58669               FIDC UNITY   2025-03-20
62367             FIDC TRACTOR   2025-03-20
68357                 FIDC 3RD   2025-03-20
80120             FIDC 3G BANK   2025-03-20
81965               FIDC BETTA   2025-03-20
84303              FIDC KERDOS   2025-03-20
87095               FIDC VELSO   2025-03-20
97707             FIDC TOPCRED   2025-03-20
1016704               FIDC SDL   2025-03-20
1017869           FIDC F2 BANK   2025-03-20
1029286          FIDC CREDILOG   2025-03-20
1029287            FIDC LOGFIN   2025-03-20
1030174          FIDC MF GROUP   2025-03-20
1039625          FIDC B ROCKET   2025-03-20
1042520          FIDC AGROCETE   2025-03-20
1042684           FIDC CREDIAL   2025-03-20
1042716            FIDC VISHNU  

In [15]:
df_estoque.NmFundo.drop_duplicates().count()

67

In [16]:
print(df_estoque.DocFundo.drop_duplicates().head(70).to_string())

0          29.109.316/0001-06
1005       30.142.432/0001-06
2568       35.940.459/0001-50
5094       36.399.603/0001-55
5230       36.544.636/0001-41
58669      38.137.056/0001-65
62367      38.137.103/0001-70
68357      40.975.325/0001-04
80120      41.387.466/0001-79
81965      41.579.569/0001-30
84303      42.462.558/0001-39
87095      42.479.713/0001-20
97707      43.181.488/0001-03
1016704    43.290.912/0001-58
1017869    43.722.793/0001-65
1029286    44.302.374/0001-37
1029287    46.268.912/0001-40
1030174    47.424.724/0001-26
1039625    47.862.139/0001-08
1042520    48.348.391/0001-66
1042684    48.928.987/0001-35
1042716    48.984.429/0001-97
1042876    49.000.456/0001-40
1043979    49.250.454/0001-00
1049106    51.430.054/0001-28
1051685    51.655.558/0001-46
1054373    51.774.918/0001-29
1054499    51.901.421/0001-24
1054555    52.041.044/0001-63
1059684    52.258.255/0001-52
1066973    52.531.368/0001-80
1068889    52.532.399/0001-55
1069535    52.549.758/0001-87
1069590   

In [17]:
# Filtrar linhas com pelo menos um NaN
df_na= df_estoque[df_estoque.isna().any(axis=1)]
df_na

,NmFundo,DocFundo,NmCedente,DocCedente,NmSacado,DocSacado,Operacao,NuDocumento,TpRecebivel,VlrNominal,VlrPresente,VlrAquisicao,VlrPDD,FaixaPDD,DtReferencia,DtVencimento,DtVencimentoAjustada,DtEmissao,DtAquisicao,Coobrigacao
1049645,FIDC BRAVOS,51.430.054/0001-28,TSI SOLUCOES LTDA,56.070.635/0001-10,tsi solucoes ltda,56.070.635/0001-10,00007514/0002471893,NaN,Nota Comercial,405612.90,352596.84,337249.34,0.0,A,2025-03-20,2025-12-19,2025-12-19,2024-12-19,2024-12-19,Nao
1069579,FIDC BOAZ,52.549.758/0001-87,ACRE COMERCIO E ADMINISTRACAO LTDA(MATRIZ),63.605.653/0001-14,ACRE COMERCIO E ADMINISTRACAO LTDA,63.605.653/0001-14,NaN,0142873800,Nota Comercial,650104.45,629528.99,620253.53,0.0,A,2025-03-20,2025-05-19,2025-05-19,2025-02-19,2025-02-20,Nao
1220780,FIDC EJM,57.941.435/0001-58,COMPANHIA SECURITIZADORA X S.A.,51.362.852/0001-60,companhia securitizadora x s.a.,51.362.852/0001-60,NaN,0000001262,Debentures,300000.00,300000.00,300000.00,0.0,A,2025-03-20,2027-11-23,2027-11-23,2025-02-25,2025-02-28,Nao


___________________

# Valida quantidade de fundos SQL

In [18]:
import mysql.connector
import pandas as pd
 
# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_CORPORATIVO',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
) 

if connection.is_connected():
    print('Conectado ao MySQL')

    # Definir a consulta SQL
    query = "SELECT NmFundo, DocFundo FROM Dm_Fundo WHERE Status = 'Ativo'"

    # Executar a consulta e obter os resultados
    cursor = connection.cursor()
    cursor.execute(query)
    resultados = cursor.fetchall()

    # Criar um DataFrame com os resultados
    colunas = [i[0] for i in cursor.description]
    Dm_Fundos = pd.DataFrame(resultados, columns=colunas)

    # Fechar o cursor
    cursor.close()

else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

Conectado ao MySQL


In [19]:
estoque = df_estoque[["DocFundo","NmFundo"]].drop_duplicates()
estoque = estoque.rename(columns={'NmFundo': 'NmFundo_Estoque'})

df_merged = pd.merge(Dm_Fundos, estoque, on='DocFundo', how='left')
df_merged = df_merged.fillna(value=0)

x = len(df_merged[df_merged["NmFundo_Estoque"]==0])
x

4

# Faltantes 

In [20]:
MensagemTeams = df_merged.loc[df_merged["NmFundo_Estoque"] == 0, "NmFundo"].to_list()
print(MensagemTeams)

['FIDC APG', 'FIDC BRISTOL', 'FICFIDC CATALISE', 'FICFIDC LION']


##### TRUNCATE NA TABELA STAGING Stg_Estoque

In [ ]:
# banco_de_dados = 'DW_STAGING'
# nome_da_sua_tabela = 'Stg_Estoque'

# con = pymysql.connect(host = 'dbc-sp-1-instance-1-instance-1.ciaao5zm9beh.sa-east-1.rds.amazonaws.com', user = 'dwuser', passwd = 'MMo6u4UHKEWr', db = banco_de_dados)
# cursor = con.cursor()
# connection

# cursor.execute(f"TRUNCATE TABLE {nome_da_sua_tabela}")

# con.close()

In [21]:
# Contar valores NaN em cada coluna
nan_por_coluna = df_estoque.isna().sum()

# Exibir o resumo
print(nan_por_coluna)

NmFundo                 0
DocFundo                0
NmCedente               0
DocCedente              0
NmSacado                0
DocSacado               0
Operacao                2
NuDocumento             1
TpRecebivel             0
VlrNominal              0
VlrPresente             0
VlrAquisicao            0
VlrPDD                  0
FaixaPDD                0
DtReferencia            0
DtVencimento            0
DtVencimentoAjustada    0
DtEmissao               0
DtAquisicao             0
Coobrigacao             0
dtype: int64


### Remover valores faltantes

In [22]:
df_estoque.fillna(0, inplace=True)

## Insere na tabela STG

In [23]:
import mysql.connector
import pandas as pd
import time
start_time = time.time()

df_final = df_estoque
 
# Função para inserir um pedaço do DataFrame na tabela MySQL em lote
def insert_dataframe_to_mysql_batch(df, connection, table_name, batch_size=100000):
    cursor = connection.cursor()
    cols = ",".join([str(i) for i in df.columns.tolist()])
    
    # Dividir o DataFrame em lotes
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(x) for x in batch_df.to_numpy()]
        placeholders = ",".join(["%s"] * len(df.columns))
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
        cursor.executemany(sql, values)
        connection.commit()
    cursor.close()

# Conectar ao banco de dados MySQL
connection = mysql.connector.connect(
    host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
    database='DW_STAGING',
    user='admin',
    password='yb222KraY7PTN0jbH7P3'
)

if connection.is_connected():
    print('Conectado ao MySQL')

    # Inserir DataFrame em lotes
    insert_dataframe_to_mysql_batch(df_final, connection, 'Stg_Estoque', batch_size=10000)
    print('Concluído')
else:
    print('Falha na conexão ao MySQL')

# Fechar a conexão com o banco de dados
connection.close()

end_time = time.time()
# Calcula o tempo decorrido
elapsed_time = end_time - start_time
print(f"Tempo decorrido: {elapsed_time} segundos")

Conectado ao MySQL
Concluído
Tempo decorrido: 65.69472694396973 segundos


### Deletar os arquivos da pasta

In [ ]:
import os
 
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Estoques'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))